In [40]:
import pandas as pd
import numpy
from __future__ import division

In [41]:
df2_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AS_segment2_workflowS2_has_duration.csv",engine='python')

In [42]:
df2_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,741976|477777|732954,Send for Assessment,Send for Assessment,7/5/2019 16:30:14,741976,NaN,4,N,1. App to Assessment,Part 1 - Application ID Workflow,...,MCW,HDCW,Full Degree,AS,AS,NaN,NaN,N,3 days 21 hours 22 minutes 3 seconds,336123.0


In [43]:
numOfWorkflowProcesses = df2_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [44]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [45]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [46]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [47]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [48]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df2_WS2T[df2_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [49]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [50]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step)

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)


In [51]:
s2_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s2_t2['Num of Occurrences per step(descending)'] = list_step_count_per_key
s2_t2['Average time spends per step(descending)'] = list_step_ave_time_per_key
s2_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step,Num of Occurrences per step(descending),Average time spends per step(descending)
0,1,115,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Send for Assessment},"[(Send for Assessment, 115)]",[]
1,2,167,19 days 8 hours 11 minutes 2 seconds,10.0 days 23.0 hours 42.0 minutes 6.0 seconds,35.93%,"{OC - Created - Other, Assessment Outcome - Su...","[(Send for Assessment, 167), (Assessment Outco...","[(Send for Assessment, 19.0 days 8.0 hours 11...."
2,3,571,19 days 12 hours 27 minutes 27 seconds,14.0 days 4.0 hours 54.0 minutes 47.0 seconds,91.77%,"{Assessment Decision - Not Qualified, OC - Cre...","[(Send for Assessment, 571), (Assessment Outco...",[(Acacdemic - Assessment Decision - Not Qualif...
3,4,334,28 days 0 hours 12 minutes 43 seconds,19.0 days 5.0 hours 22.0 minutes 4.0 seconds,85.33%,"{OC - Created - Other, Assessment Decision - N...","[(Send for Assessment, 338), (Assessment Decis...","[(AR - Satisfied - advanced standing, 9.0 days..."
4,5,221,33 days 20 hours 51 minutes 12 seconds,21.0 days 0.0 hours 15.0 minutes 38.0 seconds,87.33%,"{Assessment Decision - Not Qualified, OC - Cre...","[(Assessment Decision - Qualified, 266), (Send...","[(AR - Satisfied - previous or current study, ..."
5,6,156,35 days 21 hours 27 minutes 16 seconds,26.0 days 12.0 hours 33.0 minutes 26.0 seconds,85.26%,"{OC - Created - Other, Assessment Decision - N...","[(Assessment Decision - Qualified, 219), (Send...","[(AR - Satisfied - entry points, 8.0 days 21.0..."
6,7,78,35 days 6 hours 30 minutes 32 seconds,21.0 days 23.0 hours 0.0 minutes 0.0 seconds,80.77%,"{OC - Created - Other, Assessment Decision - N...","[(Assessment Decision - Qualified, 103), (Send...","[(Put Assessment On-Hold, 7.0 days 12.0 hours ..."
7,8,55,48 days 3 hours 50 minutes 12 seconds,25.0 days 23.0 hours 5.0 minutes 5.0 seconds,83.64%,"{Assessment Decision - Not Qualified, OC - Cre...","[(Assessment Decision - Qualified, 93), (Send ...","[(AR - Satisfied - syllabus, 9.0 days 20.0 hou..."
8,9,39,42 days 22 hours 37 minutes 14 seconds,34.0 days 18.0 hours 35.0 minutes 19.0 seconds,82.05%,"{OC - Created - Other, Assessment Decision - N...","[(Assessment Decision - Qualified, 76), (Send ...","[(Send for Assessment, 7.0 days 10.0 hours 44...."
9,10,23,70 days 2 hours 30 minutes 27 seconds,22.0 days 1.0 hours 59.0 minutes 34.0 seconds,82.61%,"{OC - Created - Other, Assessment Decision - N...","[(Assessment Decision - Qualified, 38), (Send ...","[(AR - Satisfied - GAM, 8.0 days 23.0 hours 29..."


In [52]:
s2_t2.to_csv('engineering_AS_segment2_workflowS2_table.csv', index=False)






